# RGB–Rainfall Calibration Notebook

This notebook provides a **generic pipeline** for calibrating radar RGB imagery against ground station precipitation data.

**Main features:**
- Handles long historical archives (e.g., 2011–2024)
- Aggregates 2‑minute radar frames into hourly products
- Uses maximum‑intensity aggregation (suitable for extreme events)
- Builds an RGB→rainfall regression model using station data
- Produces estimated rainfall maps and saves results in `outputs/`

In [ ]:
# pip install numpy pandas matplotlib scikit-learn pillow pyproj scipy joblib tqdm

  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
Using cached tqdm-4.67.1-py3-none-any.whl (78 kB)
Note: you may need to restart the kernel to use updated packages.


## 1. Imports and configuration

In [3]:
import os, glob
import numpy as np
import pandas as pd
from PIL import Image
from pyproj import Transformer
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error
import joblib
from tqdm import tqdm
import matplotlib.pyplot as plt

# Directories
radar_dir = '../../data/radar_sumare'
stations_csv = '../../config/SurfaceStations.csv'
grid_file = 'sumare_radar_latlon_grid.npz'
outputs_dir = 'outputs'
os.makedirs(outputs_dir, exist_ok=True)

# Radar parameters
SUMARE_RADAR_LAT = -22.955139
SUMARE_RADAR_LON = -43.248278
SUMARE_RADAR_RADIUS = 138_900  # meters
RADAR_INTERVAL_MIN = 2
STATION_INTERVAL_H = 1

## 2. Utility functions

In [4]:
def load_hourly_radar(radar_dir, dt_utc, method='max', min_frames=15):
    """Aggregate 2‑minute radar PNGs into a single hourly composite.
    method='max' (default) preserves extreme intensity.
    Returns np.ndarray (H×W×3) or None if no data."""
    pattern = f"{radar_dir}/{dt_utc:%Y/%m/%d}/{dt_utc:%Y_%m_%d_%H_}*.png"
    files = sorted(glob.glob(pattern))
    if len(files) < min_frames:
        return None
    imgs = [np.array(Image.open(f)) for f in files]
    agg_func = np.max if method == 'max' else np.mean
    return agg_func(imgs, axis=0)

def latlon_to_pixel(lat, lon, lat_grid, lon_grid):
    """Return (row,col) of the closest pixel to (lat,lon)."""
    dist = np.hypot(lat_grid - lat, lon_grid - lon)
    return np.unravel_index(np.argmin(dist), dist.shape)

def get_station_rain(df, timestamp):
    """Return precipitation (mm/h) for a given UTC hour if available."""
    try:
        return df.loc[timestamp, 'precipitation']
    except KeyError:
        return np.nan

def load_station_data(parquet_path):
    df = pd.read_parquet(parquet_path)
    df.index = pd.to_datetime(df.index, utc=True)
    return df

def extract_rgb_for_station(lat, lon, lat_grid, lon_grid, image):
    row, col = latlon_to_pixel(lat, lon, lat_grid, lon_grid)
    return tuple(image[row, col, :3])

## 3. Build the calibration dataset

In [ ]:
latlon = np.load(grid_file)
lat, lon = latlon['lat'], latlon['lon']
stations = pd.read_csv(stations_csv)

records = []
for _, st in tqdm(stations.iterrows(), total=len(stations)):
    code = st['STATION_ID']
    lat_s, lon_s = st['VL_LATITUDE'], st['VL_LONGITUDE']
    parquet_path = os.path.join('stations', f'{code}.parquet')
    if not os.path.exists(parquet_path):
        continue
    df = load_station_data(parquet_path)
    for ts, row in df.iterrows():
        rain = row['precipitation']
        if np.isnan(rain):
            continue
        img = load_hourly_radar(radar_dir, ts, method='max')
        if img is None:
            continue
        rgb = extract_rgb_for_station(lat_s, lon_s, lat, lon, img)
        records.append((*rgb, rain))

calib_df = pd.DataFrame(records, columns=['R', 'G', 'B', 'rain_mm_h'])
calib_df.to_csv(os.path.join(outputs_dir, 'calibration_data.csv'), index=False)
print(f'Calibration dataset: {len(calib_df)} samples saved.')

## 4. Train and evaluate the RGB→rainfall model

In [ ]:
if len(calib_df) < 10:
    print('Warning: too few samples for training.')

X = calib_df[['R','G','B']].values
y = calib_df['rain_mm_h'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

model = RandomForestRegressor(n_estimators=200, random_state=42)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
print(f'Model performance: R²={r2:.3f}, MAE={mae:.2f} mm/h')

joblib.dump(model, os.path.join(outputs_dir, 'rgb2rain_model.joblib'))

## 5. Apply the model to a radar frame and generate rainfall map

In [ ]:
from datetime import datetime

# Example: choose a timestamp (UTC)
timestamp = datetime(2023, 11, 1, 23, 0)
img = load_hourly_radar(radar_dir, timestamp, method='max')
if img is not None:
    rgb_flat = img.reshape(-1, 3)
    rain_flat = model.predict(rgb_flat)
    rain_map = rain_flat.reshape(img.shape[:2])
    np.save(os.path.join(outputs_dir, 'rain_estimated.npy'), rain_map)
    plt.figure(figsize=(7,6))
    plt.imshow(rain_map, cmap='turbo')
    plt.colorbar(label='Estimated rainfall (mm/h)')
    plt.title('Estimated rainfall field')
    plt.axis('off')
    plt.savefig(os.path.join(outputs_dir, 'rain_estimated_map.png'), dpi=150)
    plt.show()
else:
    print('No radar data found for selected timestamp.')

## 6. Visualization and validation

In [ ]:
def plot_validation(rgb_image, rain_map, stations_df, lat_grid, lon_grid):
    fig, axes = plt.subplots(1, 2, figsize=(14,6))
    ax1, ax2 = axes
    ax1.imshow(rgb_image.astype(np.uint8))
    ax1.set_title('Radar RGB (max reflectivity)')
    ax1.axis('off')
    im = ax2.imshow(rain_map, cmap='turbo')
    ax2.set_title('Estimated rainfall (mm/h)')
    ax2.axis('off')
    fig.colorbar(im, ax=ax2, fraction=0.046, pad=0.04)
    for _, row in stations_df.iterrows():
        r, c = latlon_to_pixel(row['VL_LATITUDE'], row['VL_LONGITUDE'], lat_grid, lon_grid)
        ax2.plot(c, r, 'wo', markersize=5)
        ax2.text(c+3, r, row['STATION_ID'], color='white', fontsize=8)
    plt.tight_layout()
    plt.savefig(os.path.join(outputs_dir, 'validation_plot.png'), dpi=150)
    plt.show()

## 7. Next steps
- Extend calibration with multiple stations and events.
- Experiment with alternative models (e.g., gradient boosting, neural nets).
- Add radar masking by range to exclude out‑of‑coverage pixels.
- Compare mean vs. max aggregation sensitivity.

All outputs are stored in the `outputs/` folder.